In [1]:
import numpy as np
import pandas as pd

In [2]:
movies = pd.read_csv ('tmdb_5000_movies.csv')
credits = pd.read_csv ('tmdb_5000_credits.csv')

In [3]:
# Merging the two dataset on basis of the movie title.
movies = movies.merge (credits, on = 'title')

In [4]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [5]:
# selected columns for content-based recommendation
movies = movies [['id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]
movies.head()

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [6]:
# Data Cleaning
movies.isnull().sum()

id          0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [7]:
movies.dropna (inplace = True)

In [8]:
movies.duplicated().sum

<bound method NDFrame._add_numeric_operations.<locals>.sum of 0       False
1       False
2       False
3       False
4       False
        ...  
4804    False
4805    False
4806    False
4807    False
4808    False
Length: 4806, dtype: bool>

In [9]:
import ast

def extractTags (obj):
    list = []
    for i in ast.literal_eval(obj):
        list.append (i['name'])
    return list

movies['genres'] = movies['genres'].apply(extractTags)
movies['keywords'] = movies['keywords'].apply(extractTags)

In [10]:
def extractCastTags (obj):
    list = []
    count = 0
    for i in ast.literal_eval(obj):
        if count != 5:
            list.append (i['name'])
            count+=1
        else:
            break
    return list

movies['cast'] = movies['cast'].apply(extractCastTags)

In [11]:
def extractCrewTags (obj):
    list = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            list.append (i['name'])
            break
    return list

movies['crew'] = movies['crew'].apply(extractCrewTags)

In [12]:
movies.head()

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley, ...",[Gore Verbinski]
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",[Sam Mendes]
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman, A...",[Christopher Nolan]
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton,...",[Andrew Stanton]


In [13]:
# removing spaces from each name to avoid ambiguity
movies['genres'] = movies['genres'].apply(lambda x:[i.replace (" ", "") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace (" ", "") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace (" ", "") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace (" ", "") for i in x])

In [14]:
movies.head()

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver, ...",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley, Ste...",[GoreVerbinski]
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux, Ralp...",[SamMendes]
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman, Anne...",[ChristopherNolan]
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton, Wi...",[AndrewStanton]


In [15]:
# creating tags
movies['genres'] = movies['genres'].apply(lambda x:" ".join(x))
movies['keywords'] = movies['keywords'].apply(lambda x:" ".join(x))
movies['cast'] = movies['cast'].apply(lambda x:" ".join(x))
movies['crew'] = movies['crew'].apply(lambda x:" ".join(x))
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [16]:
movies.head()

,id,title,overview,genres,keywords,cast,crew,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy ScienceFiction,cultureclash future spacewar spacecolony socie...,SamWorthington ZoeSaldana SigourneyWeaver Step...,JamesCameron,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",Adventure Fantasy Action,ocean drugabuse exoticisland eastindiatradingc...,JohnnyDepp OrlandoBloom KeiraKnightley Stellan...,GoreVerbinski,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,Action Adventure Crime,spy basedonnovel secretagent sequel mi6 britis...,DanielCraig ChristophWaltz LéaSeydoux RalphFie...,SamMendes,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,Action Crime Drama Thriller,dccomics crimefighter terrorist secretidentity...,ChristianBale MichaelCaine GaryOldman AnneHath...,ChristopherNolan,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca...",Action Adventure ScienceFiction,basedonnovel mars medallion spacetravel prince...,TaylorKitsch LynnCollins SamanthaMorton Willem...,AndrewStanton,"John Carter is a war-weary, former military ca..."


In [17]:
# Creating new dataframe using preprocessed data
movie_df = movies[['id', 'title', 'tags']]

In [18]:
movie_df

,id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."
...,...,...,...
4804,9367,El Mariachi,El Mariachi just wants to play his guitar and ...
4805,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...
4806,231617,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic..."
4807,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...


In [19]:
movie_df['tags'] = movie_df['tags'].apply(lambda x:x.lower())

C:\Users\suntec\AppData\Local\Temp\ipykernel_700\2085141683.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_df['tags'] = movie_df['tags'].apply(lambda x:x.lower())


In [20]:
movie_df.head()

,id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."


In [21]:
!pip install nltk

In [22]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [23]:
# Stemming to remove similar feature names
def stem (text):
    list = []
    for i in text.split():
        list.append(ps.stem(i))
    return " ".join(list)
movie_df['tags'] = movie_df['tags'].apply(stem)

C:\Users\suntec\AppData\Local\Temp\ipykernel_700\3411535407.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_df['tags'] = movie_df['tags'].apply(stem)


In [24]:
# vectorization of the data
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer (max_features=5000)
movieVectors = cv.fit_transform (movie_df['tags']).toarray()

In [25]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity (movieVectors)

In [26]:
# function to recommend 5 most similar movies based on input movie  
def recommendations (movie):
    movieIdx = movie_df[movie_df['title'] == movie].index[0]
    vectorDistance = similarity[movieIdx] 
    recommendedMovies = sorted (list(enumerate(vectorDistance)), reverse = True, key=lambda x:x[1])[1:6]
    for i in recommendedMovies:
        print (movie_df.iloc[i[0]].title)

In [31]:
import pickle
pickle.dump(movies, open('movies.pkl', 'wb'))

In [32]:
pickle.dump(movie_df.to_dict(), open('movie_dict.pkl', 'wb'))

In [33]:
pickle.dump(similarity,open('similarity.pkl','wb'))